In [1]:
# def python decorate
def deco(func):
    def inner():
        print("yyyyy")
    return inner

In [3]:
@deco
def dmnatry():
    print("I have tried!!")

In [4]:
dmnatry()

yyyyy


In [5]:
# 闭包

In [6]:
def make_avg():
    series = []
    
    def average(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    return average

In [8]:
avg = make_avg()

In [9]:
avg(10)

10.0

In [10]:
avg(11)

10.5

In [11]:
avg(12)

11.0

In [12]:
avg.__code__.co_names

('append', 'sum', 'len')

In [13]:
avg.__code__.co_varnames # 查看局部变量

('new_value', 'total')

In [14]:
avg.__code__.co_freevars

('series',)

In [15]:
# 此时在闭包中count是数字，为不可变类型
def make_avg2():
    count=0
    total=0
    
    def average(new_value):
        count += 1
        total += new_value
        return total/count
    return average

In [16]:
avg = make_avg2()

In [17]:
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [23]:
# 修改count, total变量类型为自由变量,ps: nonlocal为python3.0引入
def make_avg3():
    count=0
    total=0
    
    def average(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total/count
    return average

In [24]:
avg = make_avg3()

In [25]:
avg(10)

10.0

In [26]:
avg(11)

10.5

In [58]:
import time
import functools

In [95]:
# 定义一个计算函数运行时间的装饰器
def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_list=[]
        if args:
            arg_list.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s = %r'%(k, w) for k, w in sorted(kwargs.items())]
            arg_list.append(', '.join(pairs))
        arg_str = ', '.join(arg_list)
        print('[%0.8fs] %s(%s)->%r'%(elapsed, name, arg_str, result))
        return result
    return clocked 

In [96]:
@clock
def factorial(n):
    return 1 if n<2 else n*factorial(n-1)

In [97]:
factorial(10)

[0.00000040s] factorial(1)->1
[0.00009442s] factorial(2)->2
[0.00015921s] factorial(3)->6
[0.00017462s] factorial(4)->24
[0.00018884s] factorial(5)->120
[0.00020346s] factorial(6)->720
[0.00056533s] factorial(7)->5040
[0.00060286s] factorial(8)->40320
[0.00064869s] factorial(9)->362880
[0.00070953s] factorial(10)->3628800


3628800

In [98]:
@clock
def snooze(seconds):
    time.sleep(seconds)

In [99]:
snooze(.123)

[0.12237437s] snooze(0.123)->None


In [100]:
# 介绍functools.lru_cache的使用

In [101]:
@clock
def f(n):
    if n<2:
        return n
    else:
        return f(n-1) +f(n-2)

In [102]:
f(6)

[0.00000079s] f(1)->1
[0.00000119s] f(0)->0
[0.00044405s] f(2)->1
[0.00000079s] f(1)->1
[0.00050805s] f(3)->2
[0.00000040s] f(1)->1
[0.00000040s] f(0)->0
[0.00002805s] f(2)->1
[0.00057837s] f(4)->3
[0.00000040s] f(1)->1
[0.00000040s] f(0)->0
[0.00002686s] f(2)->1
[0.00000040s] f(1)->1
[0.00005333s] f(3)->2
[0.00070598s] f(5)->5
[0.00000040s] f(1)->1
[0.00000079s] f(0)->0
[0.00007901s] f(2)->1
[0.00000040s] f(1)->1
[0.00012563s] f(3)->2
[0.00000079s] f(1)->1
[0.00000079s] f(0)->0
[0.00006084s] f(2)->1
[0.00024533s] f(4)->3
[0.00102163s] f(6)->8


8

In [103]:
@functools.lru_cache()
@clock
def f(n):
    if n<2:
        return n
    else:
        return f(n-1) +f(n-2)

In [104]:
f(6)

[0.00000040s] f(1)->1
[0.00000040s] f(0)->0
[0.00020267s] f(2)->1
[0.00025007s] f(3)->2
[0.00026509s] f(4)->3
[0.00027654s] f(5)->5
[0.00029037s] f(6)->8


8

In [116]:
#参数化clock装饰器
DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({arg_str})->{result}'
#参数化装饰工厂函数
def clock2(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*args, **kwargs):
            t0 = time.perf_counter()
            result = func(*args, **kwargs)
            elapsed = time.perf_counter() - t0
            name = func.__name__
            arg_list=[]
            if args:
                arg_list.append(', '.join(repr(arg) for arg in args))
            if kwargs:
                pairs = ['%s = %r'%(k, w) for k, w in sorted(kwargs.items())]
                arg_list.append(', '.join(pairs))
            arg_str = ', '.join(arg_list)
            print(fmt.format(**locals()))  # **locals引用局部变量
            return result
        return clocked 
    return decorate

In [117]:
@clock2()
def factorial(n):
    return 1 if n<2 else n*factorial(n-1)

In [118]:
factorial(6)

[0.00000040s] factorial(1)->1
[0.00012247s] factorial(2)->2
[0.00034647s] factorial(3)->6
[0.00039506s] factorial(4)->24
[0.00043180s] factorial(5)->120
[0.00044721s] factorial(6)->720


720